## Classification Neural Network with California Census Data

In [1]:
#initial imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#read data
census_data = pd.read_csv('census_data.csv')

In [3]:
#look at data
census_data.head()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [4]:
census_data.describe()

,age,education_num,capital_gain,capital_loss,hours_per_week
count,32561.000000,32561.000000,32561.000000,32561.000000,32561.000000
mean,38.581647,10.080679,1077.648844,87.303830,40.437456
std,13.640433,2.572720,7385.292085,402.960219,12.347429
min,17.000000,1.000000,0.000000,0.000000,1.000000
25%,28.000000,9.000000,0.000000,0.000000,40.000000
50%,37.000000,10.000000,0.000000,0.000000,40.000000
75%,48.000000,12.000000,0.000000,0.000000,45.000000
max,90.000000,16.000000,99999.000000,4356.000000,99.000000


In [5]:
census_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 14 columns):
age               32561 non-null int64
workclass         32561 non-null object
education         32561 non-null object
education_num     32561 non-null int64
marital_status    32561 non-null object
occupation        32561 non-null object
relationship      32561 non-null object
race              32561 non-null object
gender            32561 non-null object
capital_gain      32561 non-null int64
capital_loss      32561 non-null int64
hours_per_week    32561 non-null int64
native_country    32561 non-null object
income_bracket    32561 non-null object
dtypes: int64(5), object(9)
memory usage: 3.5+ MB


### Convert data we want to predict from string to integers

In [6]:
#find unique strings
census_data['income_bracket'].unique()

array([' <=50K', ' >50K'], dtype=object)

In [7]:
#create function for conversion
def label_fix(label):
    if label==' <=50K':
        return 0
    else:
        return 1

In [8]:
census_data['income_bracket'] = census_data['income_bracket'].apply(label_fix)

In [9]:
#check it worked
census_data['income_bracket'].head()

0    0
1    0
2    0
3    0
4    0
Name: income_bracket, dtype: int64

### Split data and convert columns to tensorflow feature columns

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
X = census_data.drop(['income_bracket'], axis=1)
Y = census_data['income_bracket']

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.3)

In [13]:
census_data.columns

Index(['age', 'workclass', 'education', 'education_num', 'marital_status',
       'occupation', 'relationship', 'race', 'gender', 'capital_gain',
       'capital_loss', 'hours_per_week', 'native_country', 'income_bracket'],
      dtype='object')

In [14]:
import tensorflow as tf

C:\Users\ninja\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [15]:
#convert categorical columns
gender = tf.feature_column.categorical_column_with_vocabulary_list("gender", ["Female", "Male"])
occupation = tf.feature_column.categorical_column_with_hash_bucket("occupation", hash_bucket_size=1000)
marital_status = tf.feature_column.categorical_column_with_hash_bucket("marital_status", hash_bucket_size=1000)
relationship = tf.feature_column.categorical_column_with_hash_bucket("relationship", hash_bucket_size=1000)
education = tf.feature_column.categorical_column_with_hash_bucket("education", hash_bucket_size=1000)
workclass = tf.feature_column.categorical_column_with_hash_bucket("workclass", hash_bucket_size=1000)
native_country = tf.feature_column.categorical_column_with_hash_bucket("native_country", hash_bucket_size=1000)

In [16]:
#convert numerical columns
age = tf.feature_column.numeric_column("age")
education_num = tf.feature_column.numeric_column("education_num")
capital_gain = tf.feature_column.numeric_column("capital_gain")
capital_loss = tf.feature_column.numeric_column("capital_loss")
hours_per_week = tf.feature_column.numeric_column("hours_per_week")

In [17]:
#assemble feature columns into a list
feat_cols = [gender,occupation,marital_status,relationship,education,workclass,native_country,
            age,education_num,capital_gain,capital_loss,hours_per_week]

### Create and train linear classifier model

In [18]:
#create input function for the model
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train, y=y_train
    , batch_size=10, num_epochs=1000, shuffle=True)

In [19]:
#create the model, only two possible classes
model = tf.estimator.LinearClassifier(feature_columns=feat_cols,n_classes=2)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\ninja\\AppData\\Local\\Temp\\tmp3h9i_9re', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000002387A1B7B00>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [20]:
#train the model
model.train(input_fn=input_func, steps=20000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\ninja\AppData\Local\Temp\tmp3h9i_9re\model.ckpt.
INFO:tensorflow:loss = 6.931472, step = 1
INFO:tensorflow:global_step/sec: 82.3959
INFO:tensorflow:loss = 4.571494, step = 101 (1.214 sec)
INFO:tensorflow:global_step/sec: 218.237
INFO:tensorflow:loss = 39.374752, step = 201 (0.474 sec)
INFO:tensorflow:global_step/sec: 202.67
INFO:tensorflow:loss = 6.7612267, step = 301 (0.493 sec)
INFO:tensorflow:global_step/sec: 195.293
INFO:tensorflow:loss = 1.5443609, step = 401 (0.503 sec)
INFO:tensorflow:global_step/sec: 176.203
INFO:tensorflow:loss = 66.576675, step = 501 (0.561 sec)
INFO:tensorflow:global_step/sec: 148.265
INFO:tensorflow:loss = 3.3046565, step = 601 (0.690 sec)
INFO:tensorflow:global_step/sec

INFO:tensorflow:loss = 1.778723, step = 8001 (0.656 sec)
INFO:tensorflow:global_step/sec: 162.559
INFO:tensorflow:loss = 0.89052546, step = 8101 (0.607 sec)
INFO:tensorflow:global_step/sec: 149.888
INFO:tensorflow:loss = 3.835896, step = 8201 (0.686 sec)
INFO:tensorflow:global_step/sec: 152.401
INFO:tensorflow:loss = 5.909761, step = 8301 (0.647 sec)
INFO:tensorflow:global_step/sec: 162.297
INFO:tensorflow:loss = 2.6384363, step = 8401 (0.607 sec)
INFO:tensorflow:global_step/sec: 171.778
INFO:tensorflow:loss = 65.4945, step = 8501 (0.581 sec)
INFO:tensorflow:global_step/sec: 145.297
INFO:tensorflow:loss = 6.5886264, step = 8601 (0.683 sec)
INFO:tensorflow:global_step/sec: 143.04
INFO:tensorflow:loss = 0.8363159, step = 8701 (0.707 sec)
INFO:tensorflow:global_step/sec: 141.808
INFO:tensorflow:loss = 0.7629846, step = 8801 (0.703 sec)
INFO:tensorflow:global_step/sec: 180.135
INFO:tensorflow:loss = 6.12716, step = 8901 (0.559 sec)
INFO:tensorflow:global_step/sec: 155.967
INFO:tensorflow:l

INFO:tensorflow:global_step/sec: 149.439
INFO:tensorflow:loss = 10.374667, step = 16301 (0.666 sec)
INFO:tensorflow:global_step/sec: 146.59
INFO:tensorflow:loss = 2.545399, step = 16401 (0.685 sec)
INFO:tensorflow:global_step/sec: 143.23
INFO:tensorflow:loss = 5.1292925, step = 16501 (0.702 sec)
INFO:tensorflow:global_step/sec: 153.101
INFO:tensorflow:loss = 5.317579, step = 16601 (0.647 sec)
INFO:tensorflow:global_step/sec: 160.473
INFO:tensorflow:loss = 3.712256, step = 16701 (0.621 sec)
INFO:tensorflow:global_step/sec: 162.541
INFO:tensorflow:loss = 4.7265334, step = 16801 (0.619 sec)
INFO:tensorflow:global_step/sec: 151.494
INFO:tensorflow:loss = 26.966488, step = 16901 (0.657 sec)
INFO:tensorflow:global_step/sec: 145.949
INFO:tensorflow:loss = 2.8331714, step = 17001 (0.689 sec)
INFO:tensorflow:global_step/sec: 157.193
INFO:tensorflow:loss = 8.135664, step = 17101 (0.638 sec)
INFO:tensorflow:global_step/sec: 151.937
INFO:tensorflow:loss = 3.967846, step = 17201 (0.659 sec)
INFO:te

### Predictions

In [21]:
#create prediction input function for model
pred_input_func = tf.estimator.inputs.pandas_input_fn(
    x=X_test,
    batch_size=len(X_test),
    num_epochs=1,
    shuffle = False)

In [22]:
#predict values. Creates generator of values. needs converting
pred_gen = model.predict(pred_input_func)

In [23]:
#convert generator values into a list
predict=list(pred_gen)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\ninja\AppData\Local\Temp\tmp3h9i_9re\model.ckpt-20000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [24]:
#check data. It's a dictionary
predict[0]

{'class_ids': array([0], dtype=int64),
 'classes': array([b'0'], dtype=object),
 'logistic': array([0.3867774], dtype=float32),
 'logits': array([-0.46087855], dtype=float32),
 'probabilities': array([0.6132226 , 0.38677743], dtype=float32)}

In [25]:
#grab 'class_id' values from dictionary
predictions=[]
for i in predict:
    predictions.append(i['class_ids'][0])

In [26]:
from sklearn.metrics import classification_report

In [27]:
#Evaluation
print(classification_report(y_test,predictions))

             precision    recall  f1-score   support

          0       0.84      0.93      0.88      7382
          1       0.68      0.45      0.54      2387

avg / total       0.80      0.81      0.80      9769



### Try a DNN Classifier

In [28]:
#convert categorical columns to indicator or embedded column for DNNClassifier model
emb_gender=tf.feature_column.indicator_column(gender)
emb_occupation=tf.feature_column.indicator_column(occupation)
emb_marital_status=tf.feature_column.indicator_column(marital_status)
emb_relationship=tf.feature_column.indicator_column(relationship)
emb_education=tf.feature_column.indicator_column(education)
emb_workclass = tf.feature_column.indicator_column(workclass)
emb_native_country = tf.feature_column.indicator_column(native_country)

In [29]:
#reassemble feature columns
feat_cols2= [emb_gender,emb_occupation,emb_marital_status,emb_relationship,emb_education,emb_workclass,emb_native_country,
            age,education_num,capital_gain,capital_loss,hours_per_week]

In [36]:
#create model
model2=tf.estimator.DNNClassifier(hidden_units=[13,13,13,13,13,13],
            feature_columns=feat_cols2,n_classes=2)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\ninja\\AppData\\Local\\Temp\\tmp4nr14eto', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000238055FC198>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [37]:
#train model
model2.train(input_fn=input_func,steps=100000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\ninja\AppData\Local\Temp\tmp4nr14eto\model.ckpt.
INFO:tensorflow:loss = 8.552065, step = 1
INFO:tensorflow:global_step/sec: 98.2603
INFO:tensorflow:loss = 4.6714325, step = 101 (1.023 sec)
INFO:tensorflow:global_step/sec: 173.521
INFO:tensorflow:loss = 4.039445, step = 201 (0.574 sec)
INFO:tensorflow:global_step/sec: 177.273
INFO:tensorflow:loss = 2.7358491, step = 301 (0.560 sec)
INFO:tensorflow:global_step/sec: 171.109
INFO:tensorflow:loss = 3.6238995, step = 401 (0.589 sec)
INFO:tensorflow:global_step/sec: 177.493
INFO:tensorflow:loss = 4.6211076, step = 501 (0.559 sec)
INFO:tensorflow:global_step/sec: 162.907
INFO:tensorflow:loss = 2.8110764, step = 601 (0.619 sec)
INFO:tensorflow:global_step/se

INFO:tensorflow:loss = 5.172578, step = 8001 (0.591 sec)
INFO:tensorflow:global_step/sec: 164.432
INFO:tensorflow:loss = 2.546527, step = 8101 (0.606 sec)
INFO:tensorflow:global_step/sec: 169.162
INFO:tensorflow:loss = 2.3718257, step = 8201 (0.590 sec)
INFO:tensorflow:global_step/sec: 163.358
INFO:tensorflow:loss = 5.4795027, step = 8301 (0.612 sec)
INFO:tensorflow:global_step/sec: 167.742
INFO:tensorflow:loss = 4.1625776, step = 8401 (0.597 sec)
INFO:tensorflow:global_step/sec: 170.897
INFO:tensorflow:loss = 3.8238068, step = 8501 (0.584 sec)
INFO:tensorflow:global_step/sec: 163.357
INFO:tensorflow:loss = 4.22965, step = 8601 (0.613 sec)
INFO:tensorflow:global_step/sec: 160.473
INFO:tensorflow:loss = 1.761476, step = 8701 (0.622 sec)
INFO:tensorflow:global_step/sec: 168.877
INFO:tensorflow:loss = 3.507925, step = 8801 (0.594 sec)
INFO:tensorflow:global_step/sec: 169.449
INFO:tensorflow:loss = 7.520932, step = 8901 (0.589 sec)
INFO:tensorflow:global_step/sec: 166.312
INFO:tensorflow:l

INFO:tensorflow:global_step/sec: 183.441
INFO:tensorflow:loss = 1.8969887, step = 16301 (0.545 sec)
INFO:tensorflow:global_step/sec: 180.461
INFO:tensorflow:loss = 2.2610586, step = 16401 (0.554 sec)
INFO:tensorflow:global_step/sec: 182.768
INFO:tensorflow:loss = 2.692197, step = 16501 (0.548 sec)
INFO:tensorflow:global_step/sec: 175.681
INFO:tensorflow:loss = 1.8014792, step = 16601 (0.568 sec)
INFO:tensorflow:global_step/sec: 169.181
INFO:tensorflow:loss = 7.3976755, step = 16701 (0.591 sec)
INFO:tensorflow:global_step/sec: 178.845
INFO:tensorflow:loss = 2.2696567, step = 16801 (0.559 sec)
INFO:tensorflow:global_step/sec: 181.113
INFO:tensorflow:loss = 2.642002, step = 16901 (0.552 sec)
INFO:tensorflow:global_step/sec: 182.104
INFO:tensorflow:loss = 1.7102678, step = 17001 (0.550 sec)
INFO:tensorflow:global_step/sec: 167.742
INFO:tensorflow:loss = 2.5800765, step = 17101 (0.594 sec)
INFO:tensorflow:global_step/sec: 162.679
INFO:tensorflow:loss = 2.8936293, step = 17201 (0.613 sec)
IN

INFO:tensorflow:loss = 6.3412037, step = 24501 (0.590 sec)
INFO:tensorflow:global_step/sec: 164.62
INFO:tensorflow:loss = 1.8313081, step = 24601 (0.595 sec)
INFO:tensorflow:global_step/sec: 176.434
INFO:tensorflow:loss = 1.2216332, step = 24701 (0.564 sec)
INFO:tensorflow:global_step/sec: 180.773
INFO:tensorflow:loss = 1.3055964, step = 24801 (0.553 sec)
INFO:tensorflow:global_step/sec: 184.082
INFO:tensorflow:loss = 1.4363574, step = 24901 (0.543 sec)
INFO:tensorflow:global_step/sec: 172.561
INFO:tensorflow:loss = 3.6619823, step = 25001 (0.595 sec)
INFO:tensorflow:global_step/sec: 183.016
INFO:tensorflow:loss = 5.176963, step = 25101 (0.531 sec)
INFO:tensorflow:global_step/sec: 182.888
INFO:tensorflow:loss = 2.5311635, step = 25201 (0.547 sec)
INFO:tensorflow:global_step/sec: 182.232
INFO:tensorflow:loss = 0.7756973, step = 25301 (0.553 sec)
INFO:tensorflow:global_step/sec: 184.57
INFO:tensorflow:loss = 2.630728, step = 25401 (0.541 sec)
INFO:tensorflow:global_step/sec: 181.331
INFO

INFO:tensorflow:global_step/sec: 174.638
INFO:tensorflow:loss = 1.9437081, step = 32801 (0.577 sec)
INFO:tensorflow:global_step/sec: 174.235
INFO:tensorflow:loss = 4.668029, step = 32901 (0.573 sec)
INFO:tensorflow:global_step/sec: 180.167
INFO:tensorflow:loss = 1.9763148, step = 33001 (0.554 sec)
INFO:tensorflow:global_step/sec: 173.761
INFO:tensorflow:loss = 4.3762302, step = 33101 (0.573 sec)
INFO:tensorflow:global_step/sec: 171.806
INFO:tensorflow:loss = 4.6491966, step = 33201 (0.582 sec)
INFO:tensorflow:global_step/sec: 176.362
INFO:tensorflow:loss = 1.8566926, step = 33301 (0.567 sec)
INFO:tensorflow:global_step/sec: 176.299
INFO:tensorflow:loss = 3.3102624, step = 33401 (0.567 sec)
INFO:tensorflow:global_step/sec: 170.322
INFO:tensorflow:loss = 0.5967439, step = 33501 (0.590 sec)
INFO:tensorflow:global_step/sec: 185.808
INFO:tensorflow:loss = 5.9742727, step = 33601 (0.535 sec)
INFO:tensorflow:global_step/sec: 173.805
INFO:tensorflow:loss = 3.4378972, step = 33701 (0.579 sec)
I

...

INFO:tensorflow:global_step/sec: 184.397
INFO:tensorflow:loss = 2.6428785, step = 65801 (0.543 sec)
INFO:tensorflow:global_step/sec: 193.041
INFO:tensorflow:loss = 4.6632085, step = 65901 (0.533 sec)
INFO:tensorflow:global_step/sec: 183.223
INFO:tensorflow:loss = 3.6283207, step = 66001 (0.530 sec)
INFO:tensorflow:global_step/sec: 186.522
INFO:tensorflow:loss = 3.4026394, step = 66101 (0.552 sec)
INFO:tensorflow:global_step/sec: 183.736
INFO:tensorflow:loss = 3.0802305, step = 66201 (0.529 sec)
INFO:tensorflow:global_step/sec: 187.082
INFO:tensorflow:loss = 3.857012, step = 66301 (0.535 sec)
INFO:tensorflow:global_step/sec: 181.45
INFO:tensorflow:loss = 2.5751739, step = 66401 (0.553 sec)
INFO:tensorflow:global_step/sec: 188.631
INFO:tensorflow:loss = 4.398572, step = 66501 (0.528 sec)
INFO:tensorflow:global_step/sec: 185.683
INFO:tensorflow:loss = 2.9841003, step = 66601 (0.539 sec)
INFO:tensorflow:global_step/sec: 189.597
INFO:tensorflow:loss = 4.2916965, step = 66701 (0.527 sec)
INF

INFO:tensorflow:loss = 1.8194467, step = 74001 (0.532 sec)
INFO:tensorflow:global_step/sec: 185.203
INFO:tensorflow:loss = 1.930968, step = 74101 (0.540 sec)
INFO:tensorflow:global_step/sec: 176.493
INFO:tensorflow:loss = 5.1598787, step = 74201 (0.571 sec)
INFO:tensorflow:global_step/sec: 175.237
INFO:tensorflow:loss = 2.4608984, step = 74301 (0.571 sec)
INFO:tensorflow:global_step/sec: 156.474
INFO:tensorflow:loss = 4.9318275, step = 74401 (0.638 sec)
INFO:tensorflow:global_step/sec: 171.91
INFO:tensorflow:loss = 1.1305405, step = 74501 (0.584 sec)
INFO:tensorflow:global_step/sec: 137.139
INFO:tensorflow:loss = 0.83540785, step = 74601 (0.730 sec)
INFO:tensorflow:global_step/sec: 133.835
INFO:tensorflow:loss = 4.1357384, step = 74701 (0.744 sec)
INFO:tensorflow:global_step/sec: 141.808
INFO:tensorflow:loss = 3.757278, step = 74801 (0.705 sec)
INFO:tensorflow:global_step/sec: 127.519
INFO:tensorflow:loss = 6.0044813, step = 74901 (0.784 sec)
INFO:tensorflow:global_step/sec: 150.338
IN

INFO:tensorflow:global_step/sec: 162.701
INFO:tensorflow:loss = 3.831594, step = 82301 (0.618 sec)
INFO:tensorflow:global_step/sec: 162.033
INFO:tensorflow:loss = 2.605813, step = 82401 (0.617 sec)
INFO:tensorflow:global_step/sec: 174.476
INFO:tensorflow:loss = 4.2827063, step = 82501 (0.574 sec)
INFO:tensorflow:global_step/sec: 180.787
INFO:tensorflow:loss = 4.331366, step = 82601 (0.552 sec)
INFO:tensorflow:global_step/sec: 178.185
INFO:tensorflow:loss = 1.4637687, step = 82701 (0.561 sec)
INFO:tensorflow:global_step/sec: 131.04
INFO:tensorflow:loss = 1.6390672, step = 82801 (0.766 sec)
INFO:tensorflow:global_step/sec: 116.93
INFO:tensorflow:loss = 6.1336975, step = 82901 (0.854 sec)
INFO:tensorflow:global_step/sec: 132.945
INFO:tensorflow:loss = 3.60163, step = 83001 (0.752 sec)
INFO:tensorflow:global_step/sec: 107.269
INFO:tensorflow:loss = 5.981574, step = 83101 (0.932 sec)
INFO:tensorflow:global_step/sec: 146.806
INFO:tensorflow:loss = 4.3162093, step = 83201 (0.679 sec)
INFO:ten

INFO:tensorflow:loss = 2.2039256, step = 90501 (0.544 sec)
INFO:tensorflow:global_step/sec: 178.205
INFO:tensorflow:loss = 3.389555, step = 90601 (0.562 sec)
INFO:tensorflow:global_step/sec: 179.646
INFO:tensorflow:loss = 1.5330288, step = 90701 (0.554 sec)
INFO:tensorflow:global_step/sec: 190.786
INFO:tensorflow:loss = 9.444315, step = 90801 (0.540 sec)
INFO:tensorflow:global_step/sec: 172.416
INFO:tensorflow:loss = 5.4962177, step = 90901 (0.567 sec)
INFO:tensorflow:global_step/sec: 142.202
INFO:tensorflow:loss = 2.307919, step = 91001 (0.703 sec)
INFO:tensorflow:global_step/sec: 136.029
INFO:tensorflow:loss = 2.3377347, step = 91101 (0.735 sec)
INFO:tensorflow:global_step/sec: 123.263
INFO:tensorflow:loss = 4.7487493, step = 91201 (0.812 sec)
INFO:tensorflow:global_step/sec: 170.146
INFO:tensorflow:loss = 3.9894595, step = 91301 (0.584 sec)
INFO:tensorflow:global_step/sec: 181.48
INFO:tensorflow:loss = 3.8077683, step = 91401 (0.551 sec)
INFO:tensorflow:global_step/sec: 185.916
INFO

INFO:tensorflow:global_step/sec: 171.149
INFO:tensorflow:loss = 3.679954, step = 98801 (0.584 sec)
INFO:tensorflow:global_step/sec: 175.437
INFO:tensorflow:loss = 5.9231186, step = 98901 (0.570 sec)
INFO:tensorflow:global_step/sec: 176.634
INFO:tensorflow:loss = 8.90942, step = 99001 (0.566 sec)
INFO:tensorflow:global_step/sec: 178.208
INFO:tensorflow:loss = 5.652898, step = 99101 (0.561 sec)
INFO:tensorflow:global_step/sec: 177.574
INFO:tensorflow:loss = 5.022958, step = 99201 (0.564 sec)
INFO:tensorflow:global_step/sec: 167.743
INFO:tensorflow:loss = 1.6978177, step = 99301 (0.596 sec)
INFO:tensorflow:global_step/sec: 180.461
INFO:tensorflow:loss = 3.7625427, step = 99401 (0.554 sec)
INFO:tensorflow:global_step/sec: 180.786
INFO:tensorflow:loss = 4.304511, step = 99501 (0.552 sec)
INFO:tensorflow:global_step/sec: 146.161
INFO:tensorflow:loss = 2.097902, step = 99601 (0.686 sec)
INFO:tensorflow:global_step/sec: 140.81
INFO:tensorflow:loss = 4.467755, step = 99701 (0.709 sec)
INFO:tens

In [38]:
#prediction generator
pred_gen2=model2.predict(pred_input_func)

In [39]:
#convert to list
predict2=list(pred_gen2)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\ninja\AppData\Local\Temp\tmp4nr14eto\model.ckpt-100000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [40]:
#get predictions
predictions2=[]
for i in predict2:
    predictions2.append(i['class_ids'][0])

In [41]:
#evaluation
print(classification_report(y_test,predictions2))

             precision    recall  f1-score   support

          0       0.87      0.92      0.89      7382
          1       0.70      0.59      0.64      2387

avg / total       0.83      0.84      0.83      9769

